# Deep NLP - Word Embeddings

Think back to NLP as we've understood it so far.

If we've had some luck with NLP modeling, likely with a NaiveBayes algorithm, we were able to illustrate some correlations between words and some other feature of interest.

But to whatever extent that our models were able to make connections and pick up on correlations, they did this *without any understanding of the **meaning** of the words in question*.

Let's think for a minute about words and objective meanings!

We can make sense of meaning for computational purposes by thinking about meaning in terms of similarity, i.e. thinking about meaning *holistically*.

Q. Is there any precedent for this way of thinking about meaning? <br/>
A. [Yes](https://plato.stanford.edu/entries/meaning-holism/#ArgForMeaHol)

So what will this look like for us?

*Remember cosine similarity?*

$\rightarrow$We'll have much the same idea here: Associate each word with values along particular dimensions in a multi-dimensional space. If we had a dimension for *softness*, for example, then pillows and marshmallows would score higher on it than rocks and bricks.

In [1]:
import gensim
import numpy as np
import pandas as pd

In [2]:
# Reading in the data

import json

with open('JEOPARDY_QUESTIONS1.json') as f:
    data = json.load(f)

In [3]:
# Let's check the datatype of our data

type(data)

list

In [4]:
# And the length

len(data)

216930

In [5]:
# Let's look at the first element in our list

data[0]

{'category': 'HISTORY',
 'air_date': '2004-12-31',
 'question': "'For the last 8 years of his life, Galileo was under house arrest for espousing this man's theory'",
 'value': '$200',
 'answer': 'Copernicus',
 'round': 'Jeopardy!',
 'show_number': '4680'}

In [6]:
# How many words do we have in our first question?

len(data[0]['question'])

98

In [7]:
# Let's try that again!

data[0]['question'].split(' ')

["'For",
 'the',
 'last',
 '8',
 'years',
 'of',
 'his',
 'life,',
 'Galileo',
 'was',
 'under',
 'house',
 'arrest',
 'for',
 'espousing',
 'this',
 "man's",
 "theory'"]

In [8]:
len(data[0]['question'].split(' '))

18

In [9]:
# Let's count the total number of
# clue words we have.

length = 0

for clue in data:
    length += len(clue['question'].split(' '))
    
length

3169994

## Using Word2Vec

In [10]:
import string

In [11]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [12]:
df = pd.DataFrame(data)
df['cleaned'] = df['question'].apply(
    lambda x: x.lower()
               .translate(str.maketrans('', '', string.punctuation))
               .split()
)

In [13]:
df

,air_date,answer,category,question,round,show_number,value,cleaned
0,2004-12-31,Copernicus,HISTORY,"'For the last 8 years of his life, Galileo was...",Jeopardy!,4680,$200,"[for, the, last, 8, years, of, his, life, gali..."
1,2004-12-31,Jim Thorpe,ESPN's TOP 10 ALL-TIME ATHLETES,'No. 2: 1912 Olympian; football star at Carlis...,Jeopardy!,4680,$200,"[no, 2, 1912, olympian, football, star, at, ca..."
2,2004-12-31,Arizona,EVERYBODY TALKS ABOUT IT...,'The city of Yuma in this state has a record a...,Jeopardy!,4680,$200,"[the, city, of, yuma, in, this, state, has, a,..."
3,2004-12-31,McDonald\'s,THE COMPANY LINE,"'In 1963, live on ""The Art Linkletter Show"", t...",Jeopardy!,4680,$200,"[in, 1963, live, on, the, art, linkletter, sho..."
4,2004-12-31,John Adams,EPITAPHS & TRIBUTES,"'Signer of the Dec. of Indep., framer of the C...",Jeopardy!,4680,$200,"[signer, of, the, dec, of, indep, framer, of, ..."
5,2004-12-31,the ant,3-LETTER WORDS,"'In the title of an Aesop fable, this insect s...",Jeopardy!,4680,$200,"[in, the, title, of, an, aesop, fable, this, i..."
6,2004-12-31,the Appian Way,HISTORY,'Built in 312 B.C. to link Rome & the South of...,Jeopardy!,4680,$400,"[built, in, 312, bc, to, link, rome, the, sout..."
7,2004-12-31,Michael Jordan,ESPN's TOP 10 ALL-TIME ATHLETES,'No. 8: 30 steals for the Birmingham Barons; 2...,Jeopardy!,4680,$400,"[no, 8, 30, steals, for, the, birmingham, baro..."
8,2004-12-31,Washington,EVERYBODY TALKS ABOUT IT...,"'In the winter of 1971-72, a record 1,122 inch...",Jeopardy!,4680,$400,"[in, the, winter, of, 197172, a, record, 1122,..."
9,2004-12-31,Crate & Barrel,THE COMPANY LINE,'This housewares store was named for the packa...,Jeopardy!,4680,$400,"[this, housewares, store, was, named, for, the..."


In [14]:
# Word2Vec requires that our text have the form of a list
# of 'sentences', where each sentence is itself a list of
# words. How can we put our _Jeopardy!_ clues in that shape?

text = []
for clue in data:
    sentence = clue['question'].translate(str.maketrans('','',string.punctuation)).split(' ')
    
    new_sent = []
    for word in sentence:
        new_sent.append(word.lower())
        
    text.append(new_sent)
    

In [15]:
# Let's check the new structure of our first clue

text[0]

['for',
 'the',
 'last',
 '8',
 'years',
 'of',
 'his',
 'life',
 'galileo',
 'was',
 'under',
 'house',
 'arrest',
 'for',
 'espousing',
 'this',
 'mans',
 'theory']

In [17]:
# Constructing the model is simply a matter of
# instantiating a Word2Vec object.

model = gensim.models.Word2Vec(text, sg=1)

In [18]:
# To train, call 'train()'!

model.train(text, total_examples=model.corpus_count, epochs=model.epochs)

(11336247, 15849970)

In [19]:
# Checking word  count

model.corpus_total_words

3169994

## model.wv

In [20]:
# The '.wv' attribute stores the word vectors

model.wv

In [21]:
# The vectors are keyed by the words

model.wv['child']

array([-0.32375434,  0.25363442,  0.19556898,  0.01729642,  0.40905893,
        0.22340657, -0.48667616,  0.36684588,  0.03872521,  0.0368966 ,
       -0.0370455 , -0.4101133 ,  0.07926968, -0.40919372, -0.09477634,
        0.2655209 , -0.18909374,  0.06515364, -0.17111328, -0.02246909,
       -0.37857717,  0.07827928,  0.4665071 , -0.17058037,  0.27607435,
       -0.15525855,  0.66205305,  0.907417  ,  0.05378035, -0.00711712,
       -0.3287419 ,  0.05272369, -0.3989533 ,  0.41803744,  0.10009614,
       -0.32285208,  0.1092831 ,  0.36377087, -0.7032415 , -0.28797305,
       -0.01113832,  0.37533477,  0.18096426, -0.321657  ,  0.7014054 ,
        0.35790172, -0.32673424,  0.1677772 ,  0.03148857,  0.4126226 ,
       -0.3600144 , -0.28919268, -0.4023627 ,  0.08578733,  0.12405304,
        0.18569642,  0.51810116, -0.10494452,  0.8350915 ,  0.15614595,
        0.22059268,  0.15904698, -0.42574894, -0.38157094,  0.09969996,
       -0.43120715, -0.3821894 ,  0.27374822, -0.14478482,  0.23

### model.wv methods
#### 'most_similar()' and 'similarity()'

In [22]:
model.wv.most_similar('furniture')

[('artwork', 0.7053092122077942),
 ('decorative', 0.7052143216133118),
 ('drip', 0.6876700520515442),
 ('monogram', 0.6876224279403687),
 ('paints', 0.6872237920761108),
 ('wearers', 0.6868605017662048),
 ('fastener', 0.6836376190185547),
 ('pottery', 0.6808122396469116),
 ('typing', 0.6801878809928894),
 ('bicycles', 0.678655743598938)]

In [23]:
model.wv.similarity('furniture', 'jewelry')

0.66312504

In [24]:
# What's most similar to 'cat'?

model.wv.most_similar('cat')

[('shorthaired', 0.7200498580932617),
 ('dog', 0.695000171661377),
 ('cheetah', 0.6900850534439087),
 ('terrier', 0.6860383749008179),
 ('carnivore', 0.679857611656189),
 ('mouse', 0.6788842678070068),
 ('hound', 0.6773446798324585),
 ('pup', 0.6740412712097168),
 ('rabbit', 0.6717679500579834),
 ('schnauzer', 0.6686152219772339)]

In [25]:
# Let's try the familiar example: King - Man + Woman = Queen

model.wv.most_similar(positive=['king','woman'],
                      negative=['man'], topn=10)

[('queen', 0.667770266532898),
 ('aquitaine', 0.6147293448448181),
 ('consort', 0.6060170531272888),
 ('throne', 0.6002590656280518),
 ('margrethe', 0.5975979566574097),
 ('prince', 0.5953984260559082),
 ('isabella', 0.5914052128791809),
 ('monarch', 0.5898510217666626),
 ('princess', 0.5854009389877319),
 ('iv', 0.5791540145874023)]

In [26]:
# Shakespeare

model.wv.most_similar(['shakespeare'])

[('shakespeares', 0.7407880425453186),
 ('sophocles', 0.7405676245689392),
 ('romeo', 0.709654688835144),
 ('euripides', 0.6893974542617798),
 ('moliere', 0.6886270642280579),
 ('shakespearean', 0.6829104423522949),
 ('juliet', 0.6656391620635986),
 ('falstaff', 0.6642959117889404),
 ('proteus', 0.6637790203094482),
 ('rur', 0.6602934002876282)]

In [27]:
# Greg

model.wv.most_similar(['andy'])

[('warhol', 0.7608722448348999),
 ('warhols', 0.7171903252601624),
 ('movin', 0.7138024568557739),
 ('specials', 0.709648609161377),
 ('foxx', 0.703350305557251),
 ('animator', 0.703190803527832),
 ('showbr', 0.7027022242546082),
 ('warden', 0.7023665904998779),
 ('klugman', 0.7017198801040649),
 ('newhart', 0.6916227340698242)]

In [28]:
# Washington

model.wv.most_similar(['washington'])

[('dc', 0.7926114797592163),
 ('lincoln', 0.6474660634994507),
 ('dca', 0.6408315300941467),
 ('arlington', 0.6385438442230225),
 ('dcs', 0.6233689785003662),
 ('p3', 0.6224527955055237),
 ('washingtons', 0.6179302930831909),
 ('newseum', 0.6019489765167236),
 ('jfk', 0.5984033346176147),
 ('pataki', 0.5931301712989807)]

#### 'doesnt_match()'

In [29]:
model.wv.doesnt_match(['breakfast','lunch','frog','dinner'])

'frog'

In [30]:
model.wv.doesnt_match(['tree','root','leaf','trunk','wind'])

'wind'

#### 'closer_than()'

In [31]:
# Which words are closer to 'king' than 'queen' is?

model.wv.closer_than('king','queen')

['prince',
 'emperor',
 'kings',
 'iii',
 'throne',
 'ruler',
 'iv',
 'monarch',
 'vi',
 'sultan',
 'tudor',
 'coronation',
 'constantine',
 'ix',
 'lear',
 'xiii',
 'charlemagne',
 'darius',
 'haakon',
 'alfonso',
 'consort',
 'rama',
 'xii',
 'olaf',
 'umberto',
 'ascended',
 'lionhearted',
 'courtier',
 'harald',
 'gustavus',
 'canute',
 'ethelred',
 'thutmose',
 '751',
 '1523']

#### 'distance()'

In [32]:
# For this it will make more sense to
# normalize our vectors.

model.init_sims(replace=True)

In [33]:
model.wv.distance('king', 'king')

0.0

In [34]:
model.wv.distance('joy', 'happiness')

0.42663079500198364

#### 'evaluate_word_analogies()'

Check out [this text file](https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt)!

In [35]:
relatives = model.wv.evaluate_word_analogies('https://raw.githubusercontent.com/nicholas-leonard/word2vec/master/questions-words.txt')[1][4]


/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [36]:
len(relatives['correct'])

144

In [37]:
len(relatives['incorrect'])

276

In [38]:
relatives['correct'][:5]

[('BOY', 'GIRL', 'BROTHER', 'SISTER'),
 ('BOY', 'GIRL', 'BROTHERS', 'SISTERS'),
 ('BOY', 'GIRL', 'DAD', 'MOM'),
 ('BOY', 'GIRL', 'FATHER', 'MOTHER'),
 ('BOY', 'GIRL', 'GRANDSON', 'GRANDDAUGHTER')]

In [39]:
relatives['incorrect'][:5]

[('BOY', 'GIRL', 'GRANDFATHER', 'GRANDMOTHER'),
 ('BOY', 'GIRL', 'GRANDPA', 'GRANDMA'),
 ('BOY', 'GIRL', 'GROOM', 'BRIDE'),
 ('BOY', 'GIRL', 'HUSBAND', 'WIFE'),
 ('BOY', 'GIRL', 'NEPHEW', 'NIECE')]